In [1]:
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SequentialChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [2]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_core.prompts import PromptTemplate
llm = ChatOpenAI(model = 'gpt-4.1-nano')
prompt = PromptTemplate.from_template(
    '아래 이메일 내용중 중요한 것만 추출해.\n\n---\n\n{email_conversation}'
)

# print(prompt.format(email_conversation=email_conversation))
chain = prompt | llm 

print('---출력파서 없는 요약---')
print(chain.invoke({'email_conversation': email_conversation}).content)

---출력파서 없는 요약---
1. 바이크코퍼레이션의 김철수 상무가 "ZENESIS" 자전거에 관심을 가지고 있으며, 상세 브로슈어 요청  
2. 요청 내용: 기술 사양, 배터리 성능, 디자인 정보 제공 필요  
3. 미팅 일정 제안: 1월 15일(화요일) 오전 10시, 귀사 사무실에서 만남 추진


In [ ]:
from pydantic import BaseModel, Fiel

In [11]:
from langchain.output_parsers import PydanticOutputParser

In [13]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser # 👈 이 줄을 추가하세요

# Pydantic 모델 정의
class EmailSummary(BaseModel):
    person: str = Field(description='메일 보낸 사람')
    email: str = Field(description='보낸사람의 메일주소')
    summary: str = Field(description='메일 본문 요약')
    date: str = Field(description='메일에 언급된 미팅 날짜와 시간')

# 파서 객체 생성 (이제 오류 없이 실행됩니다)
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# 파서가 생성한 포맷팅 지침 출력
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸사람의 메일주소", "title": "Email", "type": "string"}, "summary": {"description": "메일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "summary", "date"]}
```


In [6]:
prompt = PromptTemplate.from_template(
    """
너는 요약의 신 어시스턴트야. 아래 질문에 맞게 답변을 한국어로 만들어줘

질문: {question}

이메일 내용: {email_conversation}

형식: {format}
"""
)

# 프롬프트 변수들중 일부만 채우기
prompt = prompt.partial(format=parser.get_format_instructions())

NameError: name 'parser' is not defined

In [9]:
chain = prompt | llm | parser

res = chain.invoke(
    {
        'question': '이메일 내용중 중요한 내용을 추출해 줘!',
        'email_conversation': email_conversation
    }
)

res

EmailSummary(person='김철수 상무', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무는 귀사의 'ZENESIS' 자전거에 관심을 갖고 상세 브로슈어 요청과 함께 유통 전략 논의를 위해 1월 15일 오전 10시에 사무실 미팅을 제안하였습니다.", date='2024년 1월 15일 오전 10시')